In [108]:
import pandas as pd
import numpy as np

In [109]:
df = pd.read_json('../data/ember2018/test_features.jsonl',lines=True)

In [8]:
df.iloc[0]

sha256             163ced46c18ef09d8e2f0ee4b16decf74a533f22ba3b59...
md5                                 c747678f13ec94deffeab0c44481a988
appeared                                                     2018-11
label                                                              1
avclass                                                        xtrat
histogram          [54733, 8020, 5960, 5467, 5567, 4804, 4058, 52...
byteentropy        [22510, 0, 0, 0, 0, 0, 2, 6, 0, 2, 4, 0, 0, 2,...
strings            {'numstrings': 4029, 'avlength': 5.70340034748...
general            {'size': 966594, 'vsize': 2072576, 'has_debug'...
header             {'coff': {'timestamp': 1504401044, 'machine': ...
section            {'entry': '   ', 'sections': [{'name': '   ', ...
imports            {'kernel32.dll': ['lstrcpy'], 'comctl32.dll': ...
exports                                                           []
datadirectories    [{'name': 'EXPORT_TABLE', 'size': 0, 'virtual_...
Name: 0, dtype: object

In [59]:
def explode_cols_list(df,colname):
    max_len_list = df[colname].apply(len).values.max()
    newcols = [colname+"_"+str(i) for i in range(max_len_list)]
    df[newcols] = pd.DataFrame(df[colname].tolist(), index= df.index)
    df = df.drop(columns=[colname])
    return df

df = explode_cols_list(df,'histogram')
df = explode_cols_list(df,'byteentropy')

In [61]:
df = explode_cols_list(df,'datadirectories')

In [107]:
df.iloc[1]['datadirectories']
#pd.json_normalize(df.strings)

KeyError: 'datadirectories'

In [66]:
df

,sha256,md5,appeared,label,avclass,strings,general,header,section,imports,...,datadirectories_5,datadirectories_6,datadirectories_7,datadirectories_8,datadirectories_9,datadirectories_10,datadirectories_11,datadirectories_12,datadirectories_13,datadirectories_14
0,163ced46c18ef09d8e2f0ee4b16decf74a533f22ba3b59...,c747678f13ec94deffeab0c44481a988,2018-11,1,xtrat,"{'numstrings': 4029, 'avlength': 5.70340034748...","{'size': 966594, 'vsize': 2072576, 'has_debug'...","{'coff': {'timestamp': 1504401044, 'machine': ...","{'entry': ' ', 'sections': [{'name': ' ', ...","{'kernel32.dll': ['lstrcpy'], 'comctl32.dll': ...",...,"{'name': 'BASE_RELOCATION_TABLE', 'size': 0, '...","{'name': 'DEBUG', 'size': 0, 'virtual_address'...","{'name': 'ARCHITECTURE', 'size': 0, 'virtual_a...","{'name': 'GLOBAL_PTR', 'size': 0, 'virtual_add...","{'name': 'TLS_TABLE', 'size': 0, 'virtual_addr...","{'name': 'LOAD_CONFIG_TABLE', 'size': 0, 'virt...","{'name': 'BOUND_IMPORT', 'size': 0, 'virtual_a...","{'name': 'IAT', 'size': 0, 'virtual_address': 0}","{'name': 'DELAY_IMPORT_DESCRIPTOR', 'size': 0,...","{'name': 'CLR_RUNTIME_HEADER', 'size': 0, 'vir..."
1,8d32f3535d31795886bf62523ec9035d843ef0e2c7032b...,5714e1c1af7c89ff73e397398868632c,2018-11,0,,"{'numstrings': 321, 'avlength': 13.12461059190...","{'size': 45568, 'vsize': 65536, 'has_debug': 0...","{'coff': {'timestamp': 1557472081, 'machine': ...","{'entry': 'CODE', 'sections': [{'name': 'CODE'...","{'KERNEL32.dll': ['GlobalMemoryStatus', 'lstrc...",...,"{'name': 'BASE_RELOCATION_TABLE', 'size': 1800...","{'name': 'DEBUG', 'size': 0, 'virtual_address'...","{'name': 'ARCHITECTURE', 'size': 0, 'virtual_a...","{'name': 'GLOBAL_PTR', 'size': 0, 'virtual_add...","{'name': 'TLS_TABLE', 'size': 0, 'virtual_addr...","{'name': 'LOAD_CONFIG_TABLE', 'size': 0, 'virt...","{'name': 'BOUND_IMPORT', 'size': 0, 'virtual_a...","{'name': 'IAT', 'size': 0, 'virtual_address': 0}","{'name': 'DELAY_IMPORT_DESCRIPTOR', 'size': 0,...","{'name': 'CLR_RUNTIME_HEADER', 'size': 0, 'vir..."
2,485c69ab809ef9401643c3c56567640b88b19285259c1c...,de024238e59cccd67eb093071fb4b187,2018-11,1,ramnit,"{'numstrings': 1453, 'avlength': 10.8072952512...","{'size': 298486, 'vsize': 348160, 'has_debug':...","{'coff': {'timestamp': 1099778462, 'machine': ...","{'entry': '.text', 'sections': [{'name': '.tex...","{'WS2_32.dll': ['ordinal5', 'ordinal16', 'ordi...",...,"{'name': 'BASE_RELOCATION_TABLE', 'size': 3264...","{'name': 'DEBUG', 'size': 0, 'virtual_address'...","{'name': 'ARCHITECTURE', 'size': 0, 'virtual_a...","{'name': 'GLOBAL_PTR', 'size': 0, 'virtual_add...","{'name': 'TLS_TABLE', 'size': 0, 'virtual_addr...","{'name': 'LOAD_CONFIG_TABLE', 'size': 0, 'virt...","{'name': 'BOUND_IMPORT', 'size': 0, 'virtual_a...","{'name': 'IAT', 'size': 432, 'virtual_address'...","{'name': 'DELAY_IMPORT_DESCRIPTOR', 'size': 0,...","{'name': 'CLR_RUNTIME_HEADER', 'size': 0, 'vir..."
3,f9fbcd42e4dc36bec2fb093b8e7479211def628553e2ac...,9ed302dc291bd0062ac8b4aa588e9012,2018-11,0,,"{'numstrings': 607, 'avlength': 6.963756177924...","{'size': 167936, 'vsize': 262144, 'has_debug':...","{'coff': {'timestamp': 1459953544, 'machine': ...","{'entry': '.text', 'sections': [{'name': '.tex...",{},...,"{'name': 'BASE_RELOCATION_TABLE', 'size': 0, '...","{'name': 'DEBUG', 'size': 0, 'virtual_address'...","{'name': 'ARCHITECTURE', 'size': 0, 'virtual_a...","{'name': 'GLOBAL_PTR', 'size': 0, 'virtual_add...","{'name': 'TLS_TABLE', 'size': 24, 'virtual_add...","{'name': 'LOAD_CONFIG_TABLE', 'size': 0, 'virt...","{'name': 'BOUND_IMPORT', 'size': 0, 'virtual_a...","{'name': 'IAT', 'size': 532, 'virtual_address'...","{'name': 'DELAY_IMPORT_DESCRIPTOR', 'size': 0,...","{'name': 'CLR_RUNTIME_HEADER', 'size': 0, 'vir..."
4,f5bbd3a215e4e62e12ad43b6e05e3bbae2ae5632910cbd...,035c0ebe04d3803aae1f36f02c810441,2018-11,0,,"{'numstrings': 5347, 'avlength': 18.2244249111...","{'size': 817518, 'vsize': 3952640, 'has_debug'...","{'coff': {'timestamp': 708992537, 'machine': '...","{'entry'